In [1]:
import numpy as np

In [2]:
from lsst.sims.catalogs.generation.db import CatalogDBObject
from lsst.sims.catalogs.generation.db import CatalogDBObject

In [3]:
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.sims.catUtils.baseCatalogModels import GalaxyTileObj
from lsst.sims.catalogs.measures.instance import InstanceCatalog
from lsst.sims.utils import ObservationMetaData



/usr/local/manual/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want

In [4]:
from lsst.sims.catUtils.baseCatalogModels import SNObj

In [5]:
s  = SNObj()

In [6]:
names = s.get_column_names()

/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geopoint'
  return fn(*arg, **kw)
/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geocenter'
  return fn(*arg, **kw)
/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geobox'
  return fn(*arg, **kw)


In [22]:
gTile = GalaxyTileObj()
class GTileCopy(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'galtileid','redshift']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e'}

In [7]:
for key in names:
    if 'snra' in names[key]:
        print key

TwinkSN


In [8]:
print names

{'sfdAstromEasterEggs': ['ra', 'decl', 'gal_l', 'gal_b', 'versionid', 'mura', 'mudecl', 'vrad', 'parallax', 'distance', 'sedfilename', 'sedid', 'flux_scale', 'umag', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'sdssu', 'sdssg', 'sdssr', 'sdssi', 'sdssz', 'absmr', 'newSDSSu', 'newSDSSg', 'newSDSSr', 'newSDSSi', 'newSDSSz', 'ebv', 'especid', 'pop', 'type', 't', 'feh', 'logg', 'vr', 'vphi', 'vz', 'ismultiple', 'isvar', 'timescale', 'varfluxpeak', 't0', 'x', 'y', 'z', 'cx', 'cy', 'cz', 'run', 'runobjid', 'geopoint', 'varParamStr', 'fieldid', 'magNorm', 'gizisId', 'filename', 'simobjid', 'htmID', 'varsimobjid'], 'stars_mlt_part_1180': ['htmid', 'simobjid', 'ra', 'decl', 'gal_l', 'gal_b', 'versionid', 'mura', 'mudecl', 'vrad', 'parallax', 'distance', 'sedfilename', 'sedid', 'flux_scale', 'umag', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'sdssu', 'sdssg', 'sdssr', 'sdssi', 'sdssz', 'absmr', 'newSDSSu', 'newSDSSg', 'newSDSSr', 'newSDSSi', 'newSDSSz', 'ebv', 'especid', 'pop', 'type', 't', 'feh', 'log

In [16]:
class galCopy1(InstanceCatalog):
    
    column_outputs = ['raJ2000', 'decJ2000', 'snid', 'galtileid','t0', 'x0', 'x1', 'c', 'redshift', ]
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'x0':'%8e'}
                        #'x0':'%8e', 'x1':'%8e', 'c':'%8e', 't0':'%8e'}

In [10]:
degConv = np.array([1., 1./60., 1./3600.])
raConv = degConv / 24.0 * 360.
centralRA = np.dot(np.array([3., 32., 30]), raConv) #03h 32m 30s
centralDec = np.dot(np.array([-28, 6., 0.]), degConv)
patchRadius = 0.4 * np.sqrt(2) #np.dot(np.array([0.0, 10.0, 0.]), degConv)

In [11]:
TwinklesObsMetaDataSmall = ObservationMetaData(boundType='box',pointingRA=centralRA,pointingDec=centralDec,
                                          boundLength=0.167, mjd=49540.0)

In [17]:
tsmall = galCopy(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall)

In [18]:
tsmall.write_catalog('TwinklesSN.dat')

In [19]:
!wc -l TwinklesSN.dat

   72780 TwinklesSN.dat


In [20]:
!head TwinklesSN.dat

#raJ2000, decJ2000, snid, galtileid, t0, x0, x1, c, redshift
9.268218e-01, -4.856720e-01, 10000060017065964, 60017065964, 1411.2391, 1.407584e-05, 0.0295, 0.2257, 0.3758
9.268878e-01, -4.855599e-01, 20000060005969546, 60005969546, 1435.7223, 3.749880e-06, 0.6676, 0.0428, 0.7170
9.268903e-01, -4.855595e-01, 60005969546, 60005969546, 3565.6371, 5.852821e-06, 1.7729, -0.0086, 0.7170
9.268899e-01, -4.855593e-01, 10000060005969546, 60005969546, 814.2388, 6.643831e-06, 0.8840, -0.1400, 0.7170
9.268669e-01, -4.856174e-01, 60001136957, 60001136957, 3265.2190, 1.454223e-05, -0.6587, -0.0267, 0.4383
9.268499e-01, -4.855875e-01, 60002107287, 60002107287, 3534.2897, 1.120626e-05, -0.4971, -0.1220, 0.6374
9.268275e-01, -4.855740e-01, 60000604100, 60000604100, 1372.0648, 2.843585e-06, -1.8810, 0.1141, 0.6501
9.268242e-01, -4.855792e-01, 10000060000604100, 60000604100, 1804.9731, 8.206371e-06, -0.1808, -0.0332, 0.6501
9.268235e-01, -4.855779e-01, 30000060000604100, 60000604100, 1342.9544, 9.895945e-0

In [21]:
np.radians(53.1029753341)

0.92682176218538093

In [25]:
gsmall = GTileCopy(db_obj=gTile, obs_metadata=TwinklesObsMetaDataSmall)

In [26]:
gsmall.write_catalog('gsmall.cat')

In [27]:
9.268184e-01

0.016176032648065813

In [28]:
np.radians(53.1027826951)

0.92681840000056304